## Preprocessing

In [119]:
# CODE PROVIDED
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [120]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME','SPECIAL_CONSIDERATIONS','USE_CASE'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,1,0,5000,1
1,T3,Independent,C2000,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Trust,1,100000-499999,142590,1


In [121]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
unique_counts

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [122]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [123]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Set a cutoff value
cutoff_value = 500

# Find application types to be replaced
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [124]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [125]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_greater_than_one = classification_counts[classification_counts > 1]
classification_counts_greater_than_one

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [126]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Set a cutoff value
cutoff_value_cls = 1000

# Find classifications to be replaced
classifications_to_replace = classification_counts_greater_than_one[classification_counts_greater_than_one < cutoff_value_cls].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2235
C3000     1918
C2100     1883
C1370        1
C1236        1
C2190        1
C4200        1
C5200        1
C1248        1
C2600        1
C1580        1
C6100        1
C1820        1
C1900        1
C2570        1
C1570        1
C3700        1
C2500        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120        1
C8210        1
C4500        1
C1245        1
C2561        1
C2150        1
Name: count, dtype: int64

In [127]:
# Remove outliers from the 'ASK_AMT' column for optimization 

# Calculate Q1 and Q3
Q1 = application_df['ASK_AMT'].quantile(0.25)
Q3 = application_df['ASK_AMT'].quantile(0.75)

# Calculate the IQR
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = application_df[(application_df['ASK_AMT'] < lower_bound) | (application_df['ASK_AMT'] > upper_bound)]

print(outliers['ASK_AMT'])

# Remove outliers
application_df = application_df[(application_df['ASK_AMT'] >= lower_bound) & (application_df['ASK_AMT'] <= upper_bound)]


1          108590
4          142590
6           31452
7         7508025
8           94389
           ...   
34280      401661
34281       56261
34284     3384856
34289       20197
34298    36500179
Name: ASK_AMT, Length: 8206, dtype: int64


In [128]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df)
application_df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
3,1,6692,1,False,False,False,True,False,False,False,...,True,False,False,True,False,False,False,False,False,False
5,1,5000,1,False,False,False,True,False,False,False,...,True,True,False,False,False,False,False,False,False,False
9,1,5000,0,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False


In [129]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop(['IS_SUCCESSFUL'], axis=1).values  # Features
y = application_df_encoded['IS_SUCCESSFUL'].values  # Target

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Display the shapes of the training and testing sets
print("Training Features Shape:", X_train.shape)
print("Training Target Shape:", y_train.shape)
print("Testing Features Shape:", X_test.shape)
print("Testing Target Shape:", y_test.shape)

Training Features Shape: (19569, 56)
Training Target Shape: (19569,)
Testing Features Shape: (6524, 56)
Testing Target Shape: (6524,)


In [130]:
# CODE PROVIDED
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [161]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# For optimization another hidden layer was added and the neurons per layer were adjusted

hidden_nodes_layer1 = 12
hidden_nodes_layer2 = 6
hidden_nodes_layer3 = 16

nn = tf.keras.models.Sequential()

# Input layer (specify the updated number of input features)
number_input_features = X_train_scaled.shape[1]  # Update this to match the number of features in your input data

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_61 (Dense)            (None, 12)                684       
                                                                 
 dense_62 (Dense)            (None, 6)                 78        
                                                                 
 dense_63 (Dense)            (None, 16)                112       
                                                                 
 dense_64 (Dense)            (None, 1)                 17        
                                                                 
Total params: 891 (3.48 KB)
Trainable params: 891 (3.48 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [162]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])

In [163]:
# Train the model
# For optimization epochs were decreased from 100 to 50
fit_model = nn.fit(X_train_scaled, y_train, epochs=50
                   )

Epoch 1/50
612/612 [==============================] - 2s 2ms/step - loss: 0.6008 - accuracy: 0.6971
Epoch 2/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5460 - accuracy: 0.7431
Epoch 3/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5389 - accuracy: 0.7435
Epoch 4/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5363 - accuracy: 0.7457
Epoch 5/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5350 - accuracy: 0.7461
Epoch 6/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5339 - accuracy: 0.7463
Epoch 7/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5332 - accuracy: 0.7470
Epoch 8/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5324 - accuracy: 0.7469
Epoch 9/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5323 - accuracy: 0.7471
Epoch 10/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5323 - accuracy: 0.7481

In [164]:
# CODE PROVIDED
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

204/204 - 0s - loss: 0.5324 - accuracy: 0.7502 - 482ms/epoch - 2ms/step
Loss: 0.5324048399925232, Accuracy: 0.7501533031463623


In [166]:
# Export our model to HDF5 file
# Define the file path
filepath = "C:/Users/njlan/OneDrive/Desktop/Git_Clone/Module_21/deep-learning-challenge/AlphabetSoupCharity_Optimization.h5"

# Save the model
nn.save(filepath)
